In [47]:
# 1
import os
import json
from requests import post, exceptions

api_key = "tnt_2DYrMSogwqi_Wkezkg3VyLWJJP0JcH8kleQ6InzrbF1eITbfNcs8OSx"
file_path = "collected.txt"
                                                                
try:
    with open(file_path, "rb") as f:
        print("file found")
        content = f.read()
except FileNotFoundError:
    print(f"Dosya bulunamadı: {file_path}")
    exit()

metadata = {"title": "price", "environment": "rag"}

data = {
    "metadata": json.dumps(metadata)
}

files = { "file": (file_path, content), }

headers = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/json"
}

try:
    response = post("https://api.ragie.ai/documents", data=data, files=files, headers=headers)
    response.raise_for_status()  
    print("File uploaded successfully!")
except exceptions.RequestException as e:
    print(f"Uploading error: {e}")


file found
File uploaded successfully!


In [48]:
import requests
import json

environment= "rag"
url = f"https://api.ragie.ai/documents"
headers = {"Authorization": f"Bearer {api_key}"}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()
    id = data['documents'][0]['id']
    print(f"En son yüklenen dosya idsi = {data['documents'][0]['id']}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

En son yüklenen dosya idsi = b316b02e-2a51-462a-b21b-34b6722cd5d6


In [49]:
import requests
import json



url = f"https://api.ragie.ai/documents/{id}"
headers = {"Authorization": f"Bearer {api_key}"}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  

    data = response.json()
    print(data)
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

{'id': 'b316b02e-2a51-462a-b21b-34b6722cd5d6', 'created_at': '2024-07-27T15:03:27.224003Z', 'updated_at': '2024-07-27T15:03:27.978486Z', 'status': 'partitioning', 'name': 'collected.txt', 'metadata': {'title': 'price', 'environment': 'rag'}, 'chunk_count': 0, 'external_id': None}


# our test data : material_table.csv

maybe important columns are Element,type,level and material.

Element,Element_ID,Type,Level,Material,Volume,Area,Length
Walls,348721,Generic - 200mm,Level 1,Default Wall,10.378381544244544,52.32838317982891,13500000.000000002
Walls,348799,Generic - 200mm,Level 1,Default Wall,5.924384085743169,29.87427472612334,7900000.000000001
Walls,348832,Generic - 200mm,Level 1,Default Wall,6.160371282705472,31.070456695746568,8399999.999999981
Walls,348867,Generic - 200mm,Level 1,Default Wall,2.519267765605479,12.704645393438359,3400000.0
Walls,348907,Generic - 200mm,Level 1,Default Wall,4.112708582754127,20.724919696232977,5150000.000000019
Walls,348945,Generic - 200mm,Level 1,Default Wall,3.2446830938760205,16.36313093876025,4499999.999999986
Walls,349579,Generic -100 mm,Level 1,Default Wall,1.7028762234596897,17.028762234596837,4499999.999999986
Walls,349652,Generic -100 mm,Level 1,Default Wall,2.95308057876629,29.6120357117212,7650000.000000001
Walls,350195,Generic -100 mm,Level 1,Default Wall,1.3899473195226815,13.909112479548364,3209999.9999999856
Roofs,349491,Generic - 400mm,Level 2,Default Roof,46.98353802786268,117.45884506965665,


In [50]:
def ragie(test_data):   
    filter_data = {"environment": "rag"}

    data = {"query": f"  {test_data}", "filter": filter_data,"rerank": True, "top_k": 4}
    body = json.dumps(data)

    url = "https://api.ragie.ai/retrievals"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    try:
    
        response = requests.post(url, headers=headers, data=body)
        response.raise_for_status()  

        data = response.json()
        #print(data)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    text_list = [chunk['text'] for chunk in data['scored_chunks']]
    combined_text = '\n'.join(text_list)
    return combined_text

In [54]:
import pandas as pd
df = pd.read_csv("material_table.csv")
df = df[["Element","Type"]]
test_data = df.to_json(orient="records")

In [52]:
test_data

'[{"Element":"Walls","Element_ID":348721,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wall","Volume":10.3783815442,"Area":52.3283831798,"Length":13500000.0000000019},{"Element":"Walls","Element_ID":348799,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wall","Volume":5.9243840857,"Area":29.8742747261,"Length":7900000.0000000009},{"Element":"Walls","Element_ID":348832,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wall","Volume":6.1603712827,"Area":31.0704566957,"Length":8399999.9999999814},{"Element":"Walls","Element_ID":348867,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wall","Volume":2.5192677656,"Area":12.7046453934,"Length":3400000.0},{"Element":"Walls","Element_ID":348907,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wall","Volume":4.1127085828,"Area":20.7249196962,"Length":5150000.0000000186},{"Element":"Walls","Element_ID":348945,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wal

In [53]:
ragie(test_data="wall")

''